In [15]:
from data_loader import *
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
import shutil
import pickle

class LSTM(nn.Module):
    """ A basic LSTM model. 
    """
    
    def __init__(self, in_dim, out_dim, hid_dim, batch_size, no_layers =1):
        super(LSTM, self).__init__()
        #specify the input dimensions
        self.in_dim = in_dim
        #specify the output dimensions
        self.out_dim = out_dim
        #specify hidden layer dimensions
        self.hid_dim = hid_dim
        #specify the number of layers
        self.no_layers = no_layers  
        #self.batch_size=batch_size
        
        #initialise the LSTM
        self.model = nn.LSTM(self.in_dim, self.hid_dim, self.no_layers)
        self.outputs = nn.Linear(self.hid_dim, out_dim)

    def forward(self, batch,hidden=None):
        """Pass the batch of images through each layer of the network, applying 
        """
        
        lstm_out, hidden = self.model(batch, hidden)
        y_pred = self.outputs(lstm_out.view(lstm_out.shape[0]*lstm_out.shape[1],lstm_out.shape[-1]))
        #pdb.set_trace()
        #The input is expected to contain raw, unnormalized scores for each class according to documentation
        #tag_scores = func.softmax(y_pred,dim=2)
        #return tag_scores,hidden
        return y_pred,hidden

    
def validate(val_loader, model, criterion):
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to evaluate mode
    model.eval()  
    hidden=None
    for minibatch_count, (images, labels) in enumerate(val_loader, 0):
        images=images.permute(1,0,2)
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        images=images.float()
       
        outputs,hidden = model(images,hidden)
        hidden[0].detach_() 
        hidden[1].detach_()
        
        labels=labels.view(-1)
        loss = criterion(outputs, labels)
        
        softmaxOutput=func.softmax(outputs,dim=1)
        modelDecision=torch.argmax(softmaxOutput,dim=1)
        accuracy = calculate_acc(modelDecision,labels)
        losses.update(loss.item(), labels.shape[0])
        acc.update(accuracy, labels.shape[0])

    return acc,losses
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_acc(modelDecision,labels):
    acc=torch.sum(labels==modelDecision).to(dtype=torch.float)/labels.shape[0]
    return acc

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        
        
# Setup: initialize the hyperparameters/variables
num_epochs = 100           # Number of full passes through the dataset
batch_size = 1          # Number of samples in each minibatch
learning_rate = 0.001  
#use_cuda=0
use_cuda=torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size,shuffle=False, show_sample=False,extras=extras)

# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = LSTM(in_dim=94, out_dim=94,hid_dim=100,batch_size=16,no_layers=1)
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
criterion = torch.nn.CrossEntropyLoss() #TODO - loss criteria are defined in the torch.nn package

#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Track the loss across training
total_loss = []
avg_minibatch_loss = []
best_loss = 100
acc_train_list=[]
acc_val_list=[]

loss_train_list=[]
loss_val_list=[]

hidden=None

# Begin training procedure
for epoch in range(num_epochs):
    model.train(True)
    N = 50
    N_minibatch_loss = 0.0    
    train_acc = AverageMeter()
    train_precision = AverageMeter()
    train_recall = AverageMeter()
    train_BCR = AverageMeter()
    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):

        images=images.permute(1,0,2)

        
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        images=images.float()
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()
        

        # Perform the forward pass through the network and compute the loss
        outputs,hidden = model(images,hidden)
        hidden[0].detach_() 
        hidden[1].detach_()
#         outputs.shape => batchSize * sequenceSize *dictionarySize
#         labels.shape => batchSize * sequenceSize
        labels=labels.view(-1)
        loss = criterion(outputs,labels)
        
        # Automagically compute the gradients and backpropagate the loss through the network
        #loss.backward(retain_graph=True)
        loss.backward()
        # Update the weights
        optimizer.step()

        #Calculate accuracy
        #pdb.set_trace()
        softmaxOutput=func.softmax(outputs,dim=1)
        modelDecision=torch.argmax(softmaxOutput,dim=1)
        accuracy = calculate_acc(modelDecision,labels)
        train_acc.update(accuracy, labels.shape[0])
        

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += float(loss)
        
        #TODO: Implement cross-validation
        del outputs
        del loss
        
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f, average accuracy: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss,train_acc.avg.mean()))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

            
    print("Finished", epoch + 1, "epochs of training")
    #TODO: Implement cross-validation
    with torch.no_grad():
        acc_val,loss_val = validate(val_loader, model, criterion)
    
    acc_train_list.append(train_acc.avg.mean())
    acc_val_list.append((acc_val.avg).mean())
    
    loss_val_list.append(loss_val.avg)
    print('val_loss: %.3f, average accuracy of all classes: %.3f'%(loss_val.avg,(acc_val.avg).mean()))
    
    # remember best loss and save checkpoint
    is_best = loss_val.avg <= best_loss
    best_loss = min(loss_val.avg, best_loss)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_loss': best_loss,
        'optimizer': optimizer.state_dict(),
    }, is_best)
    
print("Training complete after", epoch, "epochs")

results = { "acc_train_list": acc_train_list, "acc_val_list": acc_val_list, "loss_train_list":loss_train_list,"loss_val_list":loss_val_list}
pickle.dump( results, open( "own_results_base.p", "wb" ) )

CUDA is supported
[27 81 82 ... 93 93 93]
[81 82 63 ... 93 93 93]
[27 81 82 ... 93 93 93]
[81 82 63 ... 93 93 93]
[27 81 82 ... 93 93 93]
[81 82 63 ... 93 93 93]
Model on CUDA? True
Epoch 1, average minibatch 0 loss: 0.091, average accuracy: 0.000
Epoch 1, average minibatch 50 loss: 4.010, average accuracy: 0.133
Epoch 1, average minibatch 100 loss: 3.163, average accuracy: 0.158
Epoch 1, average minibatch 150 loss: 3.644, average accuracy: 0.155
Epoch 1, average minibatch 200 loss: 3.545, average accuracy: 0.156
Epoch 1, average minibatch 250 loss: 3.353, average accuracy: 0.165
Epoch 1, average minibatch 300 loss: 3.408, average accuracy: 0.167
Epoch 1, average minibatch 350 loss: 3.632, average accuracy: 0.167
Epoch 1, average minibatch 400 loss: 3.362, average accuracy: 0.166
Epoch 1, average minibatch 450 loss: 3.484, average accuracy: 0.161
Epoch 1, average minibatch 500 loss: 3.357, average accuracy: 0.161
Epoch 1, average minibatch 550 loss: 3.253, average accuracy: 0.160
Epoch

Epoch 2, average minibatch 2000 loss: 1.783, average accuracy: 0.494
Epoch 2, average minibatch 2050 loss: 1.374, average accuracy: 0.497
Epoch 2, average minibatch 2100 loss: 1.229, average accuracy: 0.501
Epoch 2, average minibatch 2150 loss: 1.791, average accuracy: 0.501
Epoch 2, average minibatch 2200 loss: 1.865, average accuracy: 0.501
Epoch 2, average minibatch 2250 loss: 1.704, average accuracy: 0.501
Epoch 2, average minibatch 2300 loss: 1.815, average accuracy: 0.501
Epoch 2, average minibatch 2350 loss: 1.580, average accuracy: 0.502
Epoch 2, average minibatch 2400 loss: 1.557, average accuracy: 0.503
Epoch 2, average minibatch 2450 loss: 2.018, average accuracy: 0.501
Epoch 2, average minibatch 2500 loss: 1.849, average accuracy: 0.501
Epoch 2, average minibatch 2550 loss: 1.743, average accuracy: 0.500
Epoch 2, average minibatch 2600 loss: 1.837, average accuracy: 0.499
Epoch 2, average minibatch 2650 loss: 1.706, average accuracy: 0.499
Epoch 2, average minibatch 2700 lo

Epoch 4, average minibatch 250 loss: 1.487, average accuracy: 0.515
Epoch 4, average minibatch 300 loss: 1.770, average accuracy: 0.518
Epoch 4, average minibatch 350 loss: 1.631, average accuracy: 0.525
Epoch 4, average minibatch 400 loss: 1.662, average accuracy: 0.526
Epoch 4, average minibatch 450 loss: 1.241, average accuracy: 0.540
Epoch 4, average minibatch 500 loss: 1.052, average accuracy: 0.556
Epoch 4, average minibatch 550 loss: 1.476, average accuracy: 0.557
Epoch 4, average minibatch 600 loss: 1.859, average accuracy: 0.550
Epoch 4, average minibatch 650 loss: 1.501, average accuracy: 0.551
Epoch 4, average minibatch 700 loss: 1.371, average accuracy: 0.557
Epoch 4, average minibatch 750 loss: 1.243, average accuracy: 0.563
Epoch 4, average minibatch 800 loss: 1.441, average accuracy: 0.564
Epoch 4, average minibatch 850 loss: 1.161, average accuracy: 0.571
Epoch 4, average minibatch 900 loss: 1.444, average accuracy: 0.573
Epoch 4, average minibatch 950 loss: 1.284, aver

Epoch 5, average minibatch 2400 loss: 1.223, average accuracy: 0.592
Epoch 5, average minibatch 2450 loss: 1.612, average accuracy: 0.590
Epoch 5, average minibatch 2500 loss: 1.510, average accuracy: 0.589
Epoch 5, average minibatch 2550 loss: 1.391, average accuracy: 0.589
Epoch 5, average minibatch 2600 loss: 1.548, average accuracy: 0.587
Epoch 5, average minibatch 2650 loss: 1.382, average accuracy: 0.587
Epoch 5, average minibatch 2700 loss: 1.651, average accuracy: 0.586
Epoch 5, average minibatch 2750 loss: 1.244, average accuracy: 0.586
Epoch 5, average minibatch 2800 loss: 1.387, average accuracy: 0.585
Epoch 5, average minibatch 2850 loss: 1.467, average accuracy: 0.585
Epoch 5, average minibatch 2900 loss: 1.449, average accuracy: 0.584
Epoch 5, average minibatch 2950 loss: 1.343, average accuracy: 0.584
Epoch 5, average minibatch 3000 loss: 1.540, average accuracy: 0.583
Epoch 5, average minibatch 3050 loss: 1.350, average accuracy: 0.583
Epoch 5, average minibatch 3100 lo

Epoch 7, average minibatch 650 loss: 1.181, average accuracy: 0.603
Epoch 7, average minibatch 700 loss: 1.153, average accuracy: 0.609
Epoch 7, average minibatch 750 loss: 1.094, average accuracy: 0.614
Epoch 7, average minibatch 800 loss: 1.261, average accuracy: 0.614
Epoch 7, average minibatch 850 loss: 0.994, average accuracy: 0.619
Epoch 7, average minibatch 900 loss: 1.232, average accuracy: 0.622
Epoch 7, average minibatch 950 loss: 1.140, average accuracy: 0.624
Epoch 7, average minibatch 1000 loss: 1.361, average accuracy: 0.623
Epoch 7, average minibatch 1050 loss: 1.251, average accuracy: 0.623
Epoch 7, average minibatch 1100 loss: 1.357, average accuracy: 0.622
Epoch 7, average minibatch 1150 loss: 1.514, average accuracy: 0.621
Epoch 7, average minibatch 1200 loss: 1.331, average accuracy: 0.620
Epoch 7, average minibatch 1250 loss: 1.198, average accuracy: 0.622
Epoch 7, average minibatch 1300 loss: 1.269, average accuracy: 0.622
Epoch 7, average minibatch 1350 loss: 1.4

...

Epoch 94, average minibatch 1900 loss: 0.816, average accuracy: 0.710
Epoch 94, average minibatch 1950 loss: 0.583, average accuracy: 0.713
Epoch 94, average minibatch 2000 loss: 0.889, average accuracy: 0.713
Epoch 94, average minibatch 2050 loss: 0.739, average accuracy: 0.715
Epoch 94, average minibatch 2100 loss: 0.688, average accuracy: 0.716
Epoch 94, average minibatch 2150 loss: 1.064, average accuracy: 0.715
Epoch 94, average minibatch 2200 loss: 1.087, average accuracy: 0.714
Epoch 94, average minibatch 2250 loss: 0.971, average accuracy: 0.713
Epoch 94, average minibatch 2300 loss: 1.023, average accuracy: 0.713
Epoch 94, average minibatch 2350 loss: 0.857, average accuracy: 0.713
Epoch 94, average minibatch 2400 loss: 0.892, average accuracy: 0.713
Epoch 94, average minibatch 2450 loss: 1.052, average accuracy: 0.712
Epoch 94, average minibatch 2500 loss: 1.013, average accuracy: 0.711
Epoch 94, average minibatch 2550 loss: 0.919, average accuracy: 0.711
Epoch 94, average mi

Epoch 96, average minibatch 50 loss: 1.129, average accuracy: 0.670
Epoch 96, average minibatch 100 loss: 0.809, average accuracy: 0.715
Epoch 96, average minibatch 150 loss: 1.070, average accuracy: 0.703
Epoch 96, average minibatch 200 loss: 1.011, average accuracy: 0.702
Epoch 96, average minibatch 250 loss: 1.040, average accuracy: 0.698
Epoch 96, average minibatch 300 loss: 1.087, average accuracy: 0.694
Epoch 96, average minibatch 350 loss: 1.036, average accuracy: 0.695
Epoch 96, average minibatch 400 loss: 1.057, average accuracy: 0.694
Epoch 96, average minibatch 450 loss: 0.781, average accuracy: 0.702
Epoch 96, average minibatch 500 loss: 0.718, average accuracy: 0.710
Epoch 96, average minibatch 550 loss: 0.958, average accuracy: 0.709
Epoch 96, average minibatch 600 loss: 1.203, average accuracy: 0.704
Epoch 96, average minibatch 650 loss: 0.811, average accuracy: 0.707
Epoch 96, average minibatch 700 loss: 0.834, average accuracy: 0.711
Epoch 96, average minibatch 750 los

Epoch 97, average minibatch 2100 loss: 0.761, average accuracy: 0.715
Epoch 97, average minibatch 2150 loss: 1.083, average accuracy: 0.714
Epoch 97, average minibatch 2200 loss: 1.124, average accuracy: 0.712
Epoch 97, average minibatch 2250 loss: 0.992, average accuracy: 0.712
Epoch 97, average minibatch 2300 loss: 1.048, average accuracy: 0.711
Epoch 97, average minibatch 2350 loss: 0.870, average accuracy: 0.711
Epoch 97, average minibatch 2400 loss: 0.893, average accuracy: 0.711
Epoch 97, average minibatch 2450 loss: 1.063, average accuracy: 0.710
Epoch 97, average minibatch 2500 loss: 1.020, average accuracy: 0.709
Epoch 97, average minibatch 2550 loss: 0.920, average accuracy: 0.709
Epoch 97, average minibatch 2600 loss: 1.040, average accuracy: 0.708
Epoch 97, average minibatch 2650 loss: 0.873, average accuracy: 0.709
Epoch 97, average minibatch 2700 loss: 1.085, average accuracy: 0.708
Epoch 97, average minibatch 2750 loss: 0.827, average accuracy: 0.709
Epoch 97, average mi

Epoch 99, average minibatch 250 loss: 1.322, average accuracy: 0.633
Epoch 99, average minibatch 300 loss: 1.289, average accuracy: 0.632
Epoch 99, average minibatch 350 loss: 1.339, average accuracy: 0.631
Epoch 99, average minibatch 400 loss: 1.239, average accuracy: 0.632
Epoch 99, average minibatch 450 loss: 0.980, average accuracy: 0.641
Epoch 99, average minibatch 500 loss: 0.844, average accuracy: 0.652
Epoch 99, average minibatch 550 loss: 1.109, average accuracy: 0.653
Epoch 99, average minibatch 600 loss: 1.319, average accuracy: 0.649
Epoch 99, average minibatch 650 loss: 0.911, average accuracy: 0.655
Epoch 99, average minibatch 700 loss: 0.927, average accuracy: 0.660
Epoch 99, average minibatch 750 loss: 0.915, average accuracy: 0.665
Epoch 99, average minibatch 800 loss: 1.025, average accuracy: 0.666
Epoch 99, average minibatch 850 loss: 0.812, average accuracy: 0.671
Epoch 99, average minibatch 900 loss: 0.955, average accuracy: 0.674
Epoch 99, average minibatch 950 lo

Epoch 100, average minibatch 2250 loss: 0.977, average accuracy: 0.710
Epoch 100, average minibatch 2300 loss: 1.028, average accuracy: 0.709
Epoch 100, average minibatch 2350 loss: 0.856, average accuracy: 0.709
Epoch 100, average minibatch 2400 loss: 0.881, average accuracy: 0.710
Epoch 100, average minibatch 2450 loss: 1.050, average accuracy: 0.709
Epoch 100, average minibatch 2500 loss: 1.017, average accuracy: 0.708
Epoch 100, average minibatch 2550 loss: 0.912, average accuracy: 0.708
Epoch 100, average minibatch 2600 loss: 1.035, average accuracy: 0.708
Epoch 100, average minibatch 2650 loss: 0.865, average accuracy: 0.708
Epoch 100, average minibatch 2700 loss: 1.088, average accuracy: 0.707
Epoch 100, average minibatch 2750 loss: 0.823, average accuracy: 0.708
Epoch 100, average minibatch 2800 loss: 0.953, average accuracy: 0.707
Epoch 100, average minibatch 2850 loss: 0.979, average accuracy: 0.707
Epoch 100, average minibatch 2900 loss: 1.028, average accuracy: 0.707
Epoch 

In [16]:
def toAbc1(numLabel,dictionary):
    #take in onehot encoded data and transform it to abc

    dic_val_list=list(dictionary.values())
    dic_key_list=list(dictionary.keys()) 
    abc=dic_key_list[dic_val_list.index(numLabel)]
    #can be commented    
    return abc

In [43]:
dictionary=pickle.load(open("dictionary_new.pkl","rb"))
# from bidict import bidict 
# reverse_dict = bidict(dictionary)
# reverse_dict.inverse

def sample(net, size, prime, dictionary):
        
    if(use_cuda):
        computing_device = torch.device("cuda")
    else:
        computing_device = torch.device("cpu")
    net = net.to(computing_device)
    net.eval() 
    
    h = None
    chars=list(prime)
    lis=[]
    for ch in chars:
        chNum=dictionary[ch]
        lis.append(chNum)
    
    lis=np.array(lis)
    softmaxLayer=torch.nn.Softmax()
    #pdb.set_trace()
    #a=torch.tensor(np.squeeze(np.eye(94)[lis.reshape(-1)]).reshape([len(prime), 1, 94])).to(computing_device).float()
    #pdb.set_trace()
    inputPrim=torch.tensor(fromNumLabelToOneHot(lis,dictionary).reshape([len(prime), 1, 94])).to(computing_device).float()
    with torch.no_grad():
        char, h = net(inputPrim[:-1,:], None )        
        softmaxOutput=func.softmax(char,dim=1)
        p_p=softmaxOutput.cpu()
        p_p = np.array(p_p)
        
        #pdb.set_trace()
         
        for j in range(size):           
            char,h=net(inputPrim[-1,:].view(1,1,94),h)
            softmaxOutput=func.softmax(char,dim=1)
            p_p=softmaxOutput.cpu()
            p_p = np.array(p_p) 
            Val=np.random.choice(94, 1,p=p_p.reshape(94,))
            chars.append(toAbc1(Val,dictionary))
    return chars
        
result=(sample(model, 2000, '<start>\nX:19\nT:Dusty Miller, The\nR:hop jig\nD:Tommy Keane: The Piper\'s Apron\nZ:id:hn-slipjig-19\nM:9/8', dictionary))

In [12]:
with open('generated.txt', 'w') as f:
    for item in result:
        f.write("%s" % item)

In [52]:
'<start>\nX:19\nT:Dusty Miller, The\nR:hop jig\nD:Tommy Keane: The Piper\'s Apron\nZ:id:hn-slipjig-19\nM:9/8'

"<start>\nX:19\nT:Dusty Miller, The\nR:hop jig\nD:Tommy Keane: The Piper's Apron\nZ:id:hn-slipjig-19\nM:9/8"

In [ ]:
'<start>\nX:19\nT:Dusty Miller, The\nR:hop jig\nD:Tommy Keane: The Piper's Apron\nZ:id:hn-slipjig-19\nM:9/8'

In [44]:
''.join(result)

'<start>\nX:19\nT:Dusty Miller, The\nR:phh\n\n2\n21attta1atatttftttatatIttt4tftt4atatttttttIttt2ttttIatttaNttttttItIttttat4tta atatIttIttttttataIata6atatttttt ttaIttttataaatata4atttt tt I4ttItaatatattttltttIItttata4ataaatttattattttttata4atatattttttttttataaaatahatatata ltattttIata ItItIttttttaatttttatatatatt tttIa atatatatatttt tttttttEtIaIt aIttatttItItaIt ttatYtattttttatIttttttttatI ttttttttatatatat9tta9ttat t ttatatttatattaatltttlta atatttttata attt tttatatatItaIttttttataatttIttttttatttttttYtatatataIatatatattta YtttaIItatatttatatttttt tItIltItutlt1tttttttttYtt7ttttWWWtWhhWWttttt06t}7Wth7}}hhhhW6666ttttttttttttIttaItttItat4tIt4tttt I tttaItttttttatttttttttttItattttItIt4ItItttttatt It4tttttatatItttttttatItItttItttttattttttha4ttatttltttttttatttttIIttatatIttttIttattt6tttatttatttItatttatatItttatttttttItttaIttatttttatt Ita ttttatttttttttttttt ltttttttltatttatatatttatttattta4ltttt ttaIttatttatatItItattIttttatEtItItatatatItatattttttta ItaattatatttatatatttItatltttatttatatatt4Itttaaatatttatata